In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import pyvinecopulib as pv

In [2]:
n = 1000
d = 4
p = 0.2

structure = pv.RVineStructure.simulate(d)

pair_copulas = []
for j in range(d - 1):
    tmp = []
    pair_copulas.append(tmp)
    for _ in range(d - j - 1):
        rho = np.minimum(np.maximum(np.random.beta(1, 0.75), 0.01), 0.99)
        tmp.append(pv.Bicop(family=pv.BicopFamily.gaussian, parameters=[[rho]]))

cop = pv.Vinecop(structure, pair_copulas)

U = cop.simulate(n=n, seeds=list(1 + np.arange(d)))

### Refit on subset

In [36]:
d = U.shape[1]
missing_vid = 2
orig_new_vid_map = {}
k = 0
for i in range(d):
    if (i + 1) == missing_vid:
        continue
    orig_new_vid_map[i + 1] = k + 1
    k += 1
retained_idx = [(i-1) for i in orig_new_vid_map.keys()]
orig_new_vid_map[missing_vid] = d
new_orig_vid_map = {v: k for k, v in orig_new_vid_map.items()}

In [37]:
vcop_controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])
cop2 = pv.Vinecop(data=U[:, retained_idx], controls=vcop_controls)

### Expand

In [89]:
def make_triangular_array(d):
    pair_copulas = np.empty(shape=(d-1,), dtype='object')
    for j in range(d - 1)[::-1]:
        pair_copulas[j] = list(np.empty(shape=(d-j-1,), dtype='object'))
    return list(pair_copulas)

def vfunc(fun, X1, X2, transpose=True):
    if transpose:
        return fun(np.vstack([np.array(X1), np.array(X2)]).T)
    else:
        return fun(np.vstack([np.array(X1), np.array(X2)]))

In [90]:
d_old = cop2.matrix.shape[1]
add_vid = d_old + 1
d = d_old + 1

In [91]:
pair_copulas = make_triangular_array(d)
for j in range(d_old)[::-1]:
    for i in range(d_old-j-1):
        pair_copulas[i][j+1] = cop2.get_pair_copula(i,j)

In [92]:
bcop_controls = pv.FitControlsBicop(family_set=[pv.BicopFamily.gaussian])

new_mat = np.zeros(shape=(d, d))
new_mat[:-1, 1:] = cop2.matrix

new_mat[d-1,0] = add_vid

new_mat[0,0] = new_mat[0,d-1]
idx1 = int(missing_vid - 1)
idx2 = int(new_mat[0,0] - 1)
pair_copulas[0][0] = pv.Bicop(data=U[:,[idx1, idx2]], controls=bcop_controls)

arg1 = vfunc(pair_copulas[0][0].hfunc1, U[:, idx1], U[:, idx2])
arg2 = vfunc(cop2.get_pair_copula(0,1).hfunc1, U[:, int(new_orig_vid_map[2]-1)], U[:, int(new_orig_vid_map[3]-1)])

new_mat[1,0] = 2
idx1 = int(missing_vid - 1)
idx2 = int(new_mat[1,0] - 1)
pair_copulas[1][0] = pv.Bicop(data=np.vstack([arg1, arg2]).T, controls=bcop_controls)

arg1 = vfunc(pair_copulas[1][0].hfunc1, U[:, idx1], U[:, idx2])
arg2 = vfunc(cop2.get_pair_copula(0,1).hfunc1, U[:, int(new_orig_vid_map[2]-1)], U[:, int(new_orig_vid_map[3]-1)])

new_mat[2,0] = 1
idx1 = int(missing_vid - 1)
idx2 = int(new_mat[2,0] - 1)
pair_copulas[2][0] = pv.Bicop(data=np.vstack([arg1, arg2]).T, controls=bcop_controls)

In [93]:
pair_copulas

[[<pyvinecopulib.Bicop>
  Gaussian, parameters = 0.341473,
  <pyvinecopulib.Bicop>
  Gaussian, parameters = 0.642539,
  <pyvinecopulib.Bicop>
  Gaussian, parameters = 0.472739],
 [<pyvinecopulib.Bicop>
  Gaussian, parameters = -0.291509,
  <pyvinecopulib.Bicop>
  Gaussian, parameters = -0.184769],
 [<pyvinecopulib.Bicop>
  Gaussian, parameters = 0.592602]]

In [95]:
pv.Vinecop(new_mat, pair_copulas)

<pyvinecopulib.Vinecop>
** Tree: 0
4,3 <-> Gaussian, parameters = 0.341473
1,3 <-> Gaussian, parameters = 0.642539
2,3 <-> Gaussian, parameters = 0.472739
** Tree: 1
4,2 | 3 <-> Gaussian, parameters = -0.291509
1,2 | 3 <-> Gaussian, parameters = -0.184769
** Tree: 2
4,1 | 2,3 <-> Gaussian, parameters = 0.592602